# Generation and Calibration (3compartment -> no_arm_CVS)

This notebook walks through the end-to-end workflow for generating, simulating, and calibrating a 3compartment model, then extending to a 0D-1D hybrid.

## Big Picture Aim

We want to be able to easily generate system models from CellML, tailored towards a specific use-case (number of vessels, 0D or 1D, complexity, etc), then calibrate the model to clinical data. Calibration is often difficult with a large parameter set, therefore, we use sensitivity analysis to find the most important parameters to calibrate.

## Steps overview
1. Set up imports and paths.
2. Open vessel array file for 3compartment (switch to no_arm_CVS) in PhLynx.
3. Generate CellML model from PhLynx (saved to Downloads).
4. Load the model with `SimulationHelper`.
5. Plot ground-truth data vs. uncalibrated outputs.
6. Add arm vessels in PhLynx and regenerate. (We need arms to compare against finger pressure measurement!)
7. Set the ground truth data for calibration.
8. Create a parameter identification object from Python.
9. Run sensitivity analysis over a large parameter set.
10. View sensitivity analysis plots.
11. Add an extra feature to fit to.
12. Choose the most influential parameters (e.g., top 2).
13. Run parameter identification (genetic algorithm).
14. Review calibration outputs.
15. Calculate PWV with the 0D model.
16. Switch selected vessels to 1D and generate C++ model.
17. Run the C++ model.
18. Load and plot 0D-1D hybrid results.
19. Compare PWV for 0D vs 0D-1D.
20. Concise-call hints (commented).


![optional caption](diagram_cvs_model_without_left_arm.png)

## 1) Set up imports and paths

In [ ]:
%load_ext autoreload
%autoreload 2

# Core imports
from pathlib import Path
import os
import sys

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
try:
    import opencor as oc
except:
    print('opencor not available, open this jupyter notebook with a python version that has opencor installed')
    exit()

# TODO check if needed
def series_to_constant(func):
    func.series_to_constant = True
    return func

print("Imports done")

# USER TO CHANGE: Make sure this is your path to the circualatory_autogen tutorial
project_root = Path("/home/farg967/Documents/git_projects/circulatory_autogen")
#################### TODO make this automatic in Docker ####################################

src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Set up paths
resources_dir = project_root / "resources" 
generated_models_dir = project_root / "generated_models"
param_id_output_dir = project_root / "param_id_output"
this_dir = project_root / "tutorial" / "interactive"
downloads_dir = Path.home() / "Downloads"

print("Paths done")

## 2) Open vessel array in PhLynx (manual) to visualise the model

- Open the 3compartment  vessel array in PhLynx [https:/phlynx.com].
- You could also modify, create, and rearrange modules in PhLynx (see tutorial/interactive/image_to_hemodynamics.ipynb)

## 3) Get default settings

In [ ]:
from utilities.utility_funcs import get_default_inp_data_dict
import pprint
# Model identifiers
model_name = "cvs_model"
# file_prefix = "3compartment" 
file_prefix = model_name+"_0d" 
input_param_file = f"{file_prefix}_parameters.csv"

# Base user inputs (this shows all the settings that can be changed)
inp_data_dict = get_default_inp_data_dict(file_prefix, input_param_file, resources_dir)

# TEMPORARY FOR THIS TUTORIAL
inp_data_dict['DEBUG'] = True

print('inp_data_dict set')
pprint.pprint(inp_data_dict)


In [ ]:
from scripts.script_generate_with_new_architecture import generate_with_new_architecture
import shutil

# Generate directly from resources CSVs
success = generate_with_new_architecture(inp_data_dict=inp_data_dict)
if not success:
    raise RuntimeError("Model generation failed")
else:
    print('Model generation successful')



## 4) Load the model with SimulationHelper

Use the solver wrapper to load the CellML model and prepare a simulation helper.

In [ ]:
from solver_wrappers import get_simulation_helper_from_inp_data_dict

# Simulation settings
inp_data_dict["sim_time"] = 2 # the 2 seconds we care about
inp_data_dict["pre_time"] = 20.0 # simulate for 20 seconds to get to periodic steady state

sim_helper = get_simulation_helper_from_inp_data_dict(inp_data_dict)

# Run once and plot a few representative variables
sim_helper.run()
variables_to_plot = [
    "venous_svc/u",
    "A_aorta_ascending_1/v",
    "A_aorta_ascending_1/u",
    "heart/u_rv",
]

y = sim_helper.get_results(variables_to_plot, flatten=True)
t = sim_helper.get_time()

##### Plotting #####

plot_dir = Path(param_id_output_dir) / "quicklooks"
plot_dir.mkdir(parents=True, exist_ok=True)

fig, axs = plt.subplots(2, 2, sharex=True, figsize=(8, 6))
axs = axs.flatten()
for idx, (ax, series, name) in enumerate(zip(axs, y, variables_to_plot)):
    ax.plot(t, series)
    ax.set_ylabel(name)
    ax.set_xlabel("Time [s]")

plt.tight_layout()
plt.savefig(plot_dir / "uncalibrated_outputs.png")
plt.show()

## 5) Plot ground-truth data vs uncalibrated outputs

Load the ground-truth data and compare it to the uncalibrated simulation results.

In [ ]:
# TODO: update with your ground-truth data file
# Example assumes a CSV with a time column and measurement columns

ground_truth_file = os.path.join(this_dir, "resources", "aorta_pressure_temp.txt")
if os.path.exists(ground_truth_file):
    gt = pd.read_csv(ground_truth_file, sep="\t")

    time_col = "time_s"
    var_col = "pressure_mmHg"

    time_gt = gt[time_col].to_numpy()
    pressure_pa = gt[var_col].to_numpy() * 133.322  # mmHg -> Pa

    plt.figure(figsize=(6, 4))
    plt.plot(time_gt, pressure_pa, label="ground truth")
    plt.plot(t, y[2], label="uncalibrated")
    plt.xlabel("Time [s]")
    plt.ylabel("Pressure [Pa]")
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print(f"Ground-truth file not found: {ground_truth_file}")

## 6) (SKIP THIS FOR NOW) Add arm vessels in PhLynx (manual)

- Add arm vessels in PhLynx.
- Regenerate the CellML model.
- If you exported a new CellML file, re-run the copy/generation cell above.

# 6) Create a parameter identification object which will be used to set up your calibration problem

In [ ]:
from param_id.paramID import CVS0DParamID

# first create a a param id object which will be used to set up out calibration problem
solver_info = {
    "pre_time": 1.0,
    "sim_time": 2.0,
    "dt_solver": 0.01,
}

inp_data_dict['solver_info'] = solver_info

param_id = CVS0DParamID.init_from_dict(inp_data_dict)


## 7) Set the observable data that you want to calibrate towards

In [ ]:
from utilities.obs_data_helpers import ObsDataCreator

# now create the obs data creator object for creating a dictionary that contains the data you will fit towards
obs_data_creator = ObsDataCreator()

# Protocol info (this defines your times and changes of inputs in the experiment)
pre_times = [[20]]
sim_times = [[time_gt[-1]]]
obs_dt = time_gt[1]-time_gt[0]
params_to_change = {}
obs_data_creator.add_protocol_info(pre_times, sim_times, params_to_change)

# add an entry for just fitting the mean of the pressure
entry = {
    "variable": "P aortic root mean",
    "name_for_plotting": "P_{aoMean}",
    "operands": ["A_aorta_ascending_1/u"],
    "operation": "mean",
    "unit": "Pa",
    "value": np.mean(pressure_pa),
    "std": 200, # assumed std of the mean pressure
    "obs_dt": obs_dt,
}
obs_data_creator.add_data_item(entry)
    

obs_data_dict = obs_data_creator.get_obs_data_dict()

# pprint.pprint(obs_data_dict)

# now add the obs to the param id object
param_id.set_ground_truth_data(obs_data_dict)


## 8) Create params_for_id.csv from Python

- Define the parameter bounds for identification in a CSV file. Use the same schema as `resources/3compartment_params_for_id.csv`.
- USER TASK: Write down what outputs of the CVS model (pressures? flows? features of pressure/flow traces, in which location?) you think each parameter will affect. Compare this with outputs later.


In [ ]:
# These vessels will have their stiffness calibrated
vessels_to_calibrate_stiffness = ["K_tube_A_aorta_ascending_1", "K_tube_A_aorta_ascending_2", 
                               "K_tube_A_aorta_ascending_3", "K_tube_A_aorta_ascending_4",
                               "K_tube_A_aortic_arch_1", "K_tube_A_aortic_arch_2", "K_tube_A_aortic_arch_3"]

# These vein compartments will have their compliance calibrated
vein_compartments_to_calibrate_compliance = ["venous_svc", "venous_ivc", 
                                             "venous_head_L_T","venous_head_R_T", 
                                             "venous_arm_L_T","venous_arm_R_T",
                                             "venous_leg_L_T","venous_leg_R_T",
                                             "venous_trunk_C_T"]

params_for_id_dict = [
    {
        "vessel_name": "global",
        "param_name": "q_lv_init", # we need to initialise blood volume somewhere, here we do it in the left ventricle for 
                                   # ease of implementation. This will be pumped around the body to 
                                   # give realistic volumes throughout the CVS.
                                   # Think of this as a parameter that shifts the total blood volume.
        "min": 400e-6,
        "max": 2500e-6,
        "name_for_plotting": "q_{sbv}",
    },
    {
        "vessel_name": vessels_to_calibrate_stiffness,
        "param_name": "E",  
        "min": 8e+4,
        "max": 4e+5,
        "name_for_plotting": "E_{ao}",
    },
    {
        "vessel_name": vein_compartments_to_calibrate_compliance,
        "param_name": "C",
        "min": 1e-7,
        "max": 4e-7,
        "name_for_plotting": "C_{v}",
    }
]

print(params_for_id_dict)

# now add the params to the param id object
param_id.set_params_for_id(params_for_id_dict)

# TODO the below to be implemented for easier calling. Feedback welcome.
# param_id.add_param_for_id(module_name, param_name, min, max, name_for_plotting)
# param_id.add_param_for_id(module_name, param_name, min, max, name_for_plotting)


## 9) Sensitivity analysis

Run Sobol sensitivity analysis over a large parameter set and save outputs to a defined directory.


In [ ]:
from sensitivity_analysis.sensitivityAnalysis import SensitivityAnalysis
import shutil

sa_output_dir = Path(param_id_output_dir) / "sensitivity" / file_prefix
if sa_output_dir.exists():
    shutil.rmtree(sa_output_dir) # remove the directory if it exists
sa_output_dir.mkdir(parents=True, exist_ok=True)


sa_options = {
    "method": "sobol",
    "sample_type": "saltelli",
    "num_samples": 16, # change to 256
    "output_dir": str(sa_output_dir),
}

sa_agent = SensitivityAnalysis.init_from_dict(inp_data_dict)



sa_agent.set_ground_truth_data(obs_data_dict)
sa_agent.set_params_for_id(params_for_id_dict)
sa_agent.set_sa_options(sa_options)

sa_agent.run_sensitivity_analysis(sa_options)
sa_output_dir


## 10) View the sensitivity analysis plots

In [ ]:
from IPython.display import Image, display

# Display the plots from the sensitivity analysis
plot_files = [file_path for file_path in sa_output_dir.glob("*.png")]
for plot_file in plot_files:
    display(Image(filename=str(plot_file)))


## Questions for Students

- What do you think happens if you change the ranges for your parameters? How do we ensure these ranges are valid?

## 11) Now add your own extra feature to fit to.


In [ ]:

# =====  this is an example of how to add a custom feature to the obs data creator =====
# this feature calculates the pressure at one point in time. 
# Modify this function to extract whatever feature you want! Max? Min? weights of a certain basis? Anything you want.

# Create your own feature to fit to
@series_to_constant
def my_extra_feature(time, pressure, series_output=False):
    if series_output: # needed for plotting the series in automatic plots
        return pressure
    half_idx = len(time) // 2
    return pressure[half_idx]

sa_agent.add_user_operation_func(my_extra_feature)
param_id.add_user_operation_func(my_extra_feature)

extra_entry = {
    "variable": "P aortic root half time",
    "name_for_plotting": "$P_{aoHalf}$",
    "operands": ["time","A_aorta_ascending_1/u"], # these need to correspond to the operands in my_extra_feature
    "operation": "my_extra_feature",
    "unit": "Pa",
    "value": my_extra_feature(time_gt, pressure_pa), # this uses your function to get the value you want to fit to from the ground truth data
    "std": 200, # assumed std of the pressure
}

obs_data_creator.add_data_item(extra_entry)
obs_data_dict = obs_data_creator.get_obs_data_dict()
obs_data_dict

In [ ]:

sa_agent.set_ground_truth_data(obs_data_dict)

# remove the old results before running.
if sa_output_dir.exists():
    shutil.rmtree(sa_output_dir) # remove the directory if it exists
sa_output_dir.mkdir(parents=True, exist_ok=True)

sa_agent.run_sensitivity_analysis(sa_options)



In [ ]:

# Display the plots from the sensitivity analysis
plot_files = [file_path for file_path in sa_output_dir.glob("*.png")]
for plot_file in plot_files:
    display(Image(filename=str(plot_file)))

## 12) Choose influential parameters

Review the sensitivity analysis outputs (plots/arrays) to choose the most influential parameters (e.g., top 2).


In [ ]:
# TODO: inspect SA outputs in sa_output_dir and pick top parameters
# Example placeholder list:
# most_influential_params = [
#     "global/q_lv_init"
# ]
# most_influential_params

most_influential_params = sa_agent.choose_most_impactful_params_sobol(top_n=1, index_type='ST', criterion='max', threshold=0.05, use_threshold=True)
# TODO the below needs to be done in the function
most_influential_param_names = [param.split("/")[-1] for param in most_influential_params]
most_influential_params


## 13) Parameter identification (genetic algorithm)

Use the genetic algorithm to calibrate model parameters.

$$
\theta^{\star} = \arg\min_{\theta \in \Theta} \; \sum_{i=1}^{N} w_i \left\lVert \frac{z_i(\theta) - \hat{z_i}}{\sigma_i} \right\rVert^2
$$

where $\theta^{\star}$ is the best fit parameter vector, $\Theta$ is your parameter space, defined by the min and max you set in params_for_id_dict, $z_i$ are your output features, $\hat{z_i}$ are your ground truths for those features, $\sigma_i$ are your feature standard deviations, and $w_i$ are the weights for each feature.


In [ ]:
from param_id.paramID import CVS0DParamID

inp_data_dict["param_id_method"] = "genetic_algorithm"
param_id.set_param_id_method(inp_data_dict["param_id_method"])

# Optimiser options (adjust as needed)
optimiser_options = {
    "num_calls_to_function": 100,
    "cost_convergence": 0.001,
    "max_patience": 10,
    "cost_type": "MSE",
}

param_id.set_optimiser_options(optimiser_options)
param_id.set_ground_truth_data(obs_data_dict)

# TODO the below needs to be done in the function (not visible to user)
params_for_id_subset = [
    entry for entry in params_for_id_dict
    if entry["param_name"] in most_influential_param_names
]

param_id.set_params_for_id(params_for_id_subset)

# Run calibration
param_id.run()



In [ ]:

# Plot best fit
param_id.simulate_with_best_param_vals()
param_id.plot_outputs()

## 14) Review calibration outputs

The calibration process saves plots and arrays in the parameter ID output directory. Use this section to view the plots. You can also make your own plots with the saved outputs


In [ ]:
output_dir = Path(param_id.output_dir)
print("Outputs saved to:", output_dir)

# Example: load best parameters
best_params_path = output_dir / "best_param_vals.npy"
if best_params_path.exists():
    best_params = np.load(best_params_path)
    best_params
else:
    print("best_param_vals.npy not found")

# Display the plots from the param id
plot_files = [file_path for file_path in output_dir.glob("*.png")]
for plot_file in plot_files:
    display(Image(filename=str(plot_file)))
    

# === Here you can make your own custom plots using saved outputs in output_dir ===


## STOP TO IMPROVE CALIBRATION

- We know that the arterial elastance is important for fitting pulse pressure. How would you create a feature that is sensitive to E but not to intial blood volume or venous compliance? 

- Try to improve the calibration pipeline by adding features to calibrate that aren't correlated (are close to orthogonal).

## 15) STOP HERE FOR NOW


## 16) Switch selected vessels to 1D and generate C++ model

Update vessel arrays for 1D coupling and regenerate a C++ model.


In [ ]:
vess_1d_list.extend(['A_axillary_L', 'A_brachial_L', 'A_radial_L', 'A_ulnar_L',
                     'A_superficial_palmar_arch_L_1', 'A_superficial_palmar_arch_L_2',
                     'A_comm_palmar_digital_L_1', 'A_comm_palmar_digital_L_2', 'A_comm_palmar_digital_L_3'])
    
convert_0d_to_1d(model_name_ext, resources_dir, input_param_file, folder_hyb=None, vess_1d_list=vess_1d_list)

## 17) Run the C++ model

Use `subprocess` to run the compiled binary for the 0D-1D hybrid.

In [ ]:
import subprocess

# TODO: update binary path and arguments
cpp_run_dir = Path(inp_data_dict_cpp.get("cpp_generated_models_dir", generated_models_dir / file_prefix))
cpp_binary = cpp_run_dir / "run_model"  # placeholder

# we need a step here to call the building of the cpp model
# OR WE DO THIS IN TERMINAL AND PROVIDE INSTRUCTIONS


# now run the cpp executable.
if cpp_binary.exists():
    result = subprocess.run([str(cpp_binary)], cwd=str(cpp_run_dir), capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
else:
    print(f"C++ binary not found: {cpp_binary}")


## 18) Load and plot 0D-1D hybrid results

Load the hybrid outputs and compare them to the 0D results.


In [ ]:
# TODO: update with real hybrid output file(s)
hybrid_output_csv = cpp_run_dir / "hybrid_outputs.csv"

if hybrid_output_csv.exists():
    hybrid = pd.read_csv(hybrid_output_csv)
    time_col = "time"  # TODO: update
    hybrid_var = "aortic_root/u"  # TODO: update

    plt.figure(figsize=(6, 4))
    plt.plot(t, y[2], label="0D")
    plt.plot(hybrid[time_col], hybrid[hybrid_var], label="0D-1D hybrid")
    plt.xlabel("Time [s]")
    plt.ylabel(hybrid_var)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print(f"Hybrid output not found: {hybrid_output_csv}")


## 19) PWV comparison (0D vs 0D-1D)

Compute PWV for the hybrid outputs and compare with the 0D model.


In [ ]:
# TODO: compute and compare PWV for hybrid outputs
# pwv_hybrid = compute_pwv_from_hybrid(hybrid)
# print("PWV 0D:", pwv_0d)
# print("PWV hybrid:", pwv_hybrid)
